In [332]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np 
torch.manual_seed(6)

In [333]:
x1 = autograd.Variable(torch.Tensor([[1, 2, 3], 
                                    [4, 5, 6],
                                    [7, 8, 9]]), requires_grad = True)
x2 = autograd.Variable(torch.Tensor([[1, 1, 1],
                                    [1, 1, 1],
                                    [1, 1, 1]]), requires_grad = True)
x3 = x1 + x2

In [334]:
data = [ ("me gusta comer en la cafeteria".split(), "SPANISH"),
         ("Give it to me".split(), "ENGLISH"),
         ("No creo que sea una buena idea".split(), "SPANISH"),
         ("No it is not a good idea to get lost at sea".split(), "ENGLISH") ]

test_data = [ ("Yo creo que si".split(), "SPANISH"),
              ("it is lost on me".split(), "ENGLISH")]

In [335]:
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [336]:
class DummyClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(DummyClassifier, self).__init__()

        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, bow_vec):
        out = F.softmax(self.linear(bow_vec))
        return out

In [337]:


def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])



In [338]:
model = DummyClassifier(NUM_LABELS, VOCAB_SIZE)
label_to_ix = { "SPANISH": 0, "ENGLISH": 1 }

In [339]:
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)


tensor([[0.6380, 0.3620]], grad_fn=<SoftmaxBackward0>)


C:\Users\mhfar\AppData\Local\Temp\ipykernel_15804\2323100239.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.linear(bow_vec))


In [340]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
for epoch in range(1000):
    for instance, label in data:
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))
        log_probs = model(bow_vec)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

C:\Users\mhfar\AppData\Local\Temp\ipykernel_15804\2323100239.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.linear(bow_vec))


In [343]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
print(next(model.parameters())[:,word_to_ix["creo"]])

tensor([[0.9593, 0.0407]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0226, 0.9774]], grad_fn=<SoftmaxBackward0>)
tensor([ 0.5695, -0.7037], grad_fn=<SelectBackward0>)


C:\Users\mhfar\AppData\Local\Temp\ipykernel_15804\2323100239.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.linear(bow_vec))
